# ERA5-Land Portugal heat/need exposure indices (2010–2012) — native ERA5-Land grid exports

This notebook computes four indices from **ERA5-Land HOURLY** `skin_temperature` over **Portugal (mainland + Azores + Madeira)**:

- **hdd_18**: count of days where **daily mean** skin temperature < 18°C  
- **cdd_25**: count of days where **daily mean** skin temperature > 25°C  
- **extreme_heat**: mean of the **6th–10th hottest** *daily mean* temperatures  
- **extreme_cold**: mean of the **6th–10th coldest** *daily mean* temperatures  

**Key design choice:** computations stay on the **native ERA5-Land grid** (regular lat/lon) throughout, and final exports are also on that same native grid.

Outputs:
- 4 single-band **GeoTIFFs** exported to your **Google Drive** (Earth Engine export tasks).

> Notes  
> - ERA5-Land `skin_temperature` is provided in **Kelvin**; we convert to **°C** as `K - 273.15`.  
> - Indices are computed per pixel, then clipped to Portugal.  
> - Exports run as asynchronous Earth Engine tasks; monitor them in the Earth Engine Tasks UI or via the optional polling cell.


In [ ]:
# (Colab) Install Earth Engine Python API (and helpers) if needed
!pip -q install earthengine-api geemap

In [ ]:
import ee
import datetime
import time

# Authenticate & initialize (Colab will prompt you)
try:
    ee.Initialize()
except Exception:
    ee.Authenticate()
    ee.Initialize()

print("Earth Engine initialized.")

## Parameters

In [ ]:
# ---- User parameters ----
DRIVE_FOLDER = "ERA5L_PT_nativegrid_2010_2012"  # Drive folder name to create/use
MAX_PIXELS = 1e13

START_YEAR = 2010
END_YEAR = 2012  # inclusive

ERA5_COLLECTION = "ECMWF/ERA5_LAND/HOURLY"
TEMP_BAND = "skin_temperature"  # Kelvin

# If you prefer nearest-neighbor behavior (no smoothing), set False.
USE_BILINEAR = False


## Study region: Portugal (incl. Azores + Madeira)

In [ ]:
# Portugal boundary from a built-in dataset (includes Atlantic islands)
gaul0 = ee.FeatureCollection("FAO/GAUL/2015/level0")
pt_geom = gaul0.filter(ee.Filter.eq("ADM0_NAME", "Portugal")).geometry()

# For exports it can be convenient to use a rectangle:
pt_region = pt_geom.buffer(20000).bounds()

print("Portugal geometry loaded.")
print("Approx area (km^2):", pt_geom.area().divide(1e6).getInfo())

## Determine ERA5-Land native projection (CRS + transform)

In [ ]:
# Grab the native projection from a sample ERA5-Land image.
era5 = ee.ImageCollection(ERA5_COLLECTION).select([TEMP_BAND])

sample_proj = ee.Image(era5.first()).select(TEMP_BAND).projection()

# Bring CRS + affine transform client-side for use in Export parameters.
# (Export expects plain Python types for crs/crsTransform.)
NATIVE_CRS = sample_proj.crs().getInfo()
NATIVE_TRANSFORM = sample_proj.transform().getInfo()  # 6-element list

print("Native CRS:", NATIVE_CRS)
print("Native affine transform:", NATIVE_TRANSFORM)
print("Native nominal scale (m):", sample_proj.nominalScale().getInfo())


## Build daily mean temperature collection (°C)

In [ ]:
def daily_mean_collection(start_date: ee.Date, end_date: ee.Date) -> ee.ImageCollection:
    """Daily mean (°C) from hourly ERA5-Land. end_date is exclusive."""
    n_days = end_date.difference(start_date, "day")
    day_offsets = ee.List.sequence(0, n_days.subtract(1))

    def one_day(d):
        d = ee.Number(d)
        day_start = start_date.advance(d, "day")
        day_end = day_start.advance(1, "day")

        # Hourly -> daily mean (Kelvin), then convert to °C
        img_k = era5.filterDate(day_start, day_end).mean()
        img_c = img_k.subtract(273.15).rename("tmean_c")

        return img_c.set({
            "system:time_start": day_start.millis(),
            "date_ymd": day_start.format("YYYY-MM-dd")
        })

    return ee.ImageCollection(day_offsets.map(one_day))

start_date = ee.Date.fromYMD(START_YEAR, 1, 1)
end_date = ee.Date.fromYMD(END_YEAR + 1, 1, 1)  # exclusive

daily = daily_mean_collection(start_date, end_date)
print("Daily images:", daily.size().getInfo())


## Compute yearly indices and average across 2010–2012

In [ ]:
def indices_for_year(year: ee.Number) -> ee.Image:
    y = ee.Number(year)
    y_start = ee.Date.fromYMD(y, 1, 1)
    y_end = y_start.advance(1, "year")
    dly = daily.filterDate(y_start, y_end)

    # hdd_18: count of days with daily mean < 18°C
    hdd18 = dly.map(lambda im: im.lt(18).rename("hdd_18_day")).sum().rename("hdd_18")

    # cdd_25: count of days with daily mean > 25°C
    cdd25 = dly.map(lambda im: im.gt(25).rename("cdd_25_day")).sum().rename("cdd_25")

    # Extreme heat/cold from sorted daily means.
    # toArray() yields a 2-D array [time, band] (band axis length = 1),
    # so after reducing over time we must index with [0, 0].
    arr = dly.select("tmean_c").toArray()     # [time, band]
    arr_sorted = arr.arraySort()              # ascending (coldest -> hottest)
    n = arr_sorted.arrayLength(0)

    # 6th–10th coldest: indices 5..9
    cold_slice = arr_sorted.arraySlice(0, 5, 10)
    extreme_cold = (
        cold_slice
        .arrayReduce(ee.Reducer.mean(), [0])
        .arrayGet([0, 0])
        .rename("extreme_cold")
    )

    # 6th–10th hottest: slice (n-10) .. (n-5)
    hot_start = n.subtract(10)
    hot_end = n.subtract(5)
    hot_slice = arr_sorted.arraySlice(0, hot_start, hot_end)
    extreme_heat = (
        hot_slice
        .arrayReduce(ee.Reducer.mean(), [0])
        .arrayGet([0, 0])
        .rename("extreme_heat")
    )

    return ee.Image.cat([hdd18, cdd25, extreme_heat, extreme_cold]).set({"year": y})

years = ee.List.sequence(START_YEAR, END_YEAR)
yearly = ee.ImageCollection(years.map(indices_for_year))

avg_indices = yearly.mean().clip(pt_region)

if USE_BILINEAR:
    avg_indices = avg_indices.resample("bilinear")

print("Bands:", avg_indices.bandNames().getInfo())


## Export GeoTIFFs to Google Drive (native ERA5-Land grid)

In [ ]:
def export_band(band_name: str):
    file_base = f"PT_{band_name}_ERA5L_skinT_dailyMean_{START_YEAR}_{END_YEAR}_nativegrid"
    task = ee.batch.Export.image.toDrive(
        image=avg_indices.select([band_name]).toFloat(),
        description=f"EXPORT_{file_base}",
        folder=DRIVE_FOLDER,
        fileNamePrefix=file_base,
        region=pt_region,
        crs=NATIVE_CRS,
        crsTransform=NATIVE_TRANSFORM,  # ensures native grid alignment
        maxPixels=MAX_PIXELS,
        fileFormat="GeoTIFF"
    )
    task.start()
    return task

bands = ["hdd_18", "cdd_25", "extreme_heat", "extreme_cold"]
tasks = {b: export_band(b) for b in bands}

print("Started export tasks:")
for b, t in tasks.items():
    print(f"  {b:13s}  task_id={t.id}")

## (Optional) Poll task statuses

In [ ]:
def show_status():
    for b, t in tasks.items():
        s = t.status()
        print(f"{b:13s} | {s.get('state'):10s} | {s.get('description')}")

# Poll a few times (adjust as you like)
for _ in range(6):
    show_status()
    time.sleep(10)
